In [145]:
from bs4 import BeautifulSoup
from html.parser import HTMLParser  
from urllib.request import urlopen  
from urllib import parse
import pandas as pd

In [153]:
class GoundFloor():
    def __init__(self, link):
        self.a = urlopen(link).read()
        self.soup = BeautifulSoup(self.a)
    
    def crawl(self):
        cards = soup.find_all("div", class_="card")

        loans = []
        for card in cards:
            loan = {}
            loan['name'] = card.find('div', class_="title").get_text()
            loan['type'] = card.find('div', class_='inner-arrow').get_text()[1:-2]
            loan['grade'] = card.find('div', class_="triangle").get_text()

            numbers = card.find_all('div', class_ = 'number')
            loan['rate'] = numbers[0].get_text()
            loan['projected_term'] = numbers[1].get_text()
            loan['loan_to_value'] = numbers[1].get_text()

            loan['details'] = card.find('div', class_='large-link').find('a').attrs['href']
            
            detail_link = 'https://www.groundfloor.us' + loan['details']
            details = self.crawl_details(detail_link)
            loan.update(details)
            loans.append(loan)
            
        return pd.DataFrame(loans)
    
    def crawl_details(self, detail_link):
        detail_soup = BeautifulSoup(urlopen(detail_link).read())
        
        company = detail_soup.find_all('div', class_='col-xs-11 anchor-link')[0].get_text()[1:-1]
        borrower = detail_soup.find_all('div', class_='col-xs-11 anchor-link')[1].get_text()[1:-1]
        
        loan_amount = detail_soup.find_all('div', class_='black-box')[3].get_text()
        investers = detail_soup.find_all('div', class_='black-box')[4].get_text()
        
        purpose = detail_soup.find_all('div', class_='white-box')[0].get_text()[4:-4]
        loan_position = detail_soup.find_all('div', class_='white-box')[1].get_text()
        total_loan_amount = detail_soup.find_all('div', class_='white-box')[2].get_text()
        loan_status = detail_soup.find_all('div', class_='white-box')[3].get_text()[1:-1]
        
        try:
            start_on = detail_soup.find_all('div', class_='value-in-box col-xs-7')[0].get_text()
            funded_on = detail_soup.find_all('div', class_='value-in-box col-xs-7')[1].get_text()
            repaid_on = detail_soup.find_all('div', class_='value-in-box col-xs-7')[2].get_text()
            matures_on = detail_soup.find_all('div', class_='value-in-box col-xs-7')[3].get_text()
        except IndexError:
            print('dates for {} unavailable:'.format(detail_link))
            start_on = -1
            funded_on = -1
            repaid_on =-1 
            matures_on = -1
            
        address = detail_soup.find(id = 'cucumber-investment').find('a').get_text()
        description = detail_soup.find(id = 'cucumber-investment').find('div', class_ = "col-xs-12").get_text()[1:-1]
        
        return {'company': company, 'borrower': borrower, 'loan_amount': loan_amount, \
 'investers':investers, 'purpose':purpose,'loan_position': loan_position, 'investers':investers, \
 'purpose':purpose, 'loan_position':loan_position, 'total_loan_amount':total_loan_amount, 'loan_status': loan_status, \
 'start_on': start_on, 'funded_on': funded_on, 'repaid_on':repaid_on, 'matures_on': matures_on,\
 'address':address, 'description':description}

## Funded

In [154]:
funded = GoundFloor('https://www.groundfloor.us/education/funded')
funded_df = funded.crawl()

dates for https://www.groundfloor.us/investments/4107-belle-avenue-21215 unavailable:


//anaconda/envs/py35/lib/python3.5/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [157]:
funded_df.columns

Index(['address', 'borrower', 'company', 'description', 'details', 'funded_on',
       'grade', 'investers', 'loan_amount', 'loan_position', 'loan_status',
       'loan_to_value', 'matures_on', 'name', 'projected_term', 'purpose',
       'rate', 'repaid_on', 'start_on', 'total_loan_amount', 'type'],
      dtype='object')

## Funding